In [2]:
# Install required packages
!pip install ultralytics
!pip install lap  # for Hungarian algorithm
!pip install torchreid
!pip install opencv-python
!pip install pandas
!pip install filterpy  # for Kalman filter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [3]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import pandas as pd

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def process_cvat_to_training_data(video_dir, output_base_dir):
    """
    Convert CVAT annotations to ReID training format

    Args:
        video_dir: Directory containing video and gt.txt
        output_base_dir: Where to save training crops
    """

    # File paths
    video_path = None
    gt_path = None

    # Find video and annotation files
    for file in os.listdir(video_dir):
        if file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(video_dir, file)
            # print(f"found video file {video_path}")
        elif file == 'gt' or file.endswith('gt.txt'):
            gt_path = os.path.join(video_dir, 'gt', 'gt.txt')
            # print(f"found annotation file {gt_path}")

    if not video_path:
        raise ValueError(f"No video file found in {video_dir}")

    # if not os.path.exists(gt_path):
    #     gt_path = os.path.join(video_dir, 'gt.txt')  # Alternative location

    print(f"Processing video: {video_path}")
    print(f"Using annotations: {gt_path}")

    # Read annotations
    # CVAT MOT format: frame,id,x,y,w,h,conf,class,visibility
    try:
        annotations = pd.read_csv(gt_path, header=None,
                                 names=['frame', 'id', 'x', 'y', 'w', 'h', 'conf', 'class', 'vis'])
    except:
        print(f"Error reading {gt_path}. Check file format.")
        return

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return

    # Get video name for unique player IDs
    video_name = os.path.basename(video_path).split('.')[0]

    # Create output directories for each unique player
    unique_ids = annotations['id'].unique()
    player_dirs = {}

    for pid in unique_ids:
        # Create unique player ID across all videos
        global_player_id = f"{video_name}_player_{pid:03d}"
        player_dir = os.path.join(output_base_dir, global_player_id)
        os.makedirs(player_dir, exist_ok=True)
        player_dirs[pid] = player_dir
        print(f"Created directory: {player_dir}")

    # Process frames
    frame_count = 1
    crops_extracted = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Get annotations for current frame
        frame_annotations = annotations[annotations['frame'] == frame_count]

        for _, row in frame_annotations.iterrows():
            x, y, w, h = int(row['x']), int(row['y']), int(row['w']), int(row['h'])
            player_id = int(row['id'])

            # Validate crop boundaries
            if x >= 0 and y >= 0 and x + w <= frame.shape[1] and y + h <= frame.shape[0]:
                # Extract player crop
                crop = frame[y:y+h, x:x+w]

                # Skip very small crops
                if crop.shape[0] > 30 and crop.shape[1] > 20:
                    # Save crop
                    crop_filename = f"frame_{frame_count:06d}.jpg"
                    crop_path = os.path.join(player_dirs[player_id], crop_filename)
                    cv2.imwrite(crop_path, crop)
                    crops_extracted += 1

        frame_count += 1

        # Progress indicator
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames, extracted {crops_extracted} crops")

    cap.release()
    print(f"Completed! Total crops extracted: {crops_extracted}")

    # Print summary
    for pid, directory in player_dirs.items():
        crop_count = len([f for f in os.listdir(directory) if f.endswith('.jpg')])
        print(f"Player {pid}: {crop_count} crops in {directory}")

In [6]:
def process_all_videos(reid_base_dir="/content/drive/MyDrive/FIT3163,3164/REID/train", training_output_dir="/content/reid_training_data"):
    """Process all CVAT annotated videos"""

    # Directory structure after CVAT export:
    # /content/drive/MyDrive/badminton_annotations/
    # ├── video1/
    # │   ├── video1.mp4
    # │   └── gt/gt.txt
    # ├── video2/
    # │   ├── video2.mp4
    # │   └── gt/gt.txt

    # Process each video directory
    for video_folder in os.listdir(reid_base_dir):
        video_dir_path = os.path.join(reid_base_dir, video_folder)
        if os.path.isdir(video_dir_path):
            print(f"\nProcessing {video_dir_path}...")
            process_cvat_to_training_data(video_dir_path, training_output_dir)
        else:
            print(f"{video_dir_path} not found")

In [13]:
def train_badminton_reid_model(register_dataset=False,
                               dataset_name='badminton',
                               data_dir="/content/reid_training_data",
                               test_identity_ratio=0.2,
                               min_test_ids=4,
                               query_per_id=1,
                               per_video_as_camid=True,
                               seed=42):
    """Train custom ReID model on badminton data."""
    import os
    import glob
    import random
    import numpy as np
    from collections import defaultdict
    import torch
    import torchreid

    random.seed(seed)
    np.random.seed(seed)

    # 0) Check data root
    if not os.path.isdir(data_dir):
        print(f"Error: Directory '{data_dir}' not found. Please ensure your data is at this path.")
        return

    # 1) Collect identity folders (skip hidden/system dirs)
    all_dirs = [
        d for d in os.listdir(data_dir)
        if os.path.isdir(os.path.join(data_dir, d))
        and not d.startswith('.')
        and d.lower() != 'market1501'  # avoid treating an external dataset dir as an ID
        and d != '__MACOSX'
        and d != 'lost+found'
    ]
    if len(all_dirs) == 0:
        print(f"No identity folders found under {data_dir}")
        return

    # 2) Parse identity folders into (img_path, pid, camid)
    ALLOWED_EXTS = ('.jpg', '.jpeg', '.png', '.bmp')
    pid_to_id = {}            # map identity_name -> global int pid
    camid_by_video = {}       # map video_name -> camid
    next_pid = 0
    next_cam = 0
    pid_to_imgs = defaultdict(list)  # pid -> list[(path, pid, camid)]
    total_images = 0

    def infer_video_name(identity_folder_name: str) -> str:
        """
        Heuristic to derive 'video name' from folder name to assign camid.
        e.g. 'bryan_1_player_003' -> 'bryan_1'
        Fallback: whole folder name if pattern not present.
        """
        if '_player_' in identity_folder_name:
            return identity_folder_name.split('_player_')[0]
        if '#p' in identity_folder_name.lower():
            return identity_folder_name.split('#')[0]
        return identity_folder_name  # fallback

    for ident in sorted(all_dirs):
        # Avoid hidden/temporary folders inside id dir (handled below)
        identity_dir = os.path.join(data_dir, ident)

        # Assign PID
        if ident not in pid_to_id:
            pid_to_id[ident] = next_pid
            next_pid += 1
        pid = pid_to_id[ident]

        # CamID: per video (recommended), or a single cam=0 for all
        video_name = infer_video_name(ident)
        if per_video_as_camid:
            if video_name not in camid_by_video:
                camid_by_video[video_name] = next_cam
                next_cam += 1
            camid = camid_by_video[video_name]
        else:
            camid = 0

        # Gather images
        img_paths = []
        for fn in os.listdir(identity_dir):
            if fn.startswith('.') or fn == '.ipynb_checkpoints':
                continue
            if fn.lower().endswith(ALLOWED_EXTS):
                img_paths.append(os.path.join(identity_dir, fn))

        if len(img_paths) == 0:
            print(f"Warning: identity '{ident}' has no images, skipping.")
            continue

        for p in sorted(img_paths):
            pid_to_imgs[p] = 1  # ensure path uniqueness
        for p in sorted(img_paths):
            pid_to_imgs[pid].append((p, pid, camid))
            total_images += 1

    total_identities = len(pid_to_id)
    print(f"Training data summary:")
    print(f"- Total identities: {total_identities}:")
    for i, identity in enumerate(pid_to_id):
        print(f"{i}. {identity}")
    print(f"\n- Total images: {total_images}")
    if total_identities > 0:
        print(f"- Average images per identity: {total_images/total_identities:.1f}")
    if total_identities < 10:
        print("Warning: Less than 10 identities. Consider collecting more data.")

    # 3) Split identities into train vs test (query/gallery from test IDs only)
    all_pids = list(pid_to_imgs.keys())
    # pid_to_imgs keys contain ints (pids) and also we inserted a dict of paths above by mistake—fix:
    # Ensure we only keep integer keys (the real pid keys)
    all_pids = [k for k in pid_to_imgs.keys() if isinstance(k, int)]
    random.shuffle(all_pids)

    num_test_ids = max(min_test_ids, int(round(len(all_pids) * test_identity_ratio)))
    num_test_ids = min(num_test_ids, len(all_pids))  # guard
    test_pids = set(all_pids[:num_test_ids])
    train_pids = set(all_pids[num_test_ids:])

    # Build train/query/gallery lists
    train_set_data = []
    query_data = []
    gallery_data = []

    # Training set (will be relabeled to 0..N_train-1)
    for pid in sorted(train_pids):
        train_set_data.extend(pid_to_imgs[pid])

    # Relabel training PIDs to be contiguous starting at 0
    unique_train_pids = sorted({pid for _, pid, _ in train_set_data})
    pid_remap = {pid: i for i, pid in enumerate(unique_train_pids)}
    train_set_data = [(p, pid_remap[pid], cam) for (p, pid, cam) in train_set_data]

    QUERY_CAM_OFFSET = 1000  # force query camids to differ from gallery camids

    # Test set: for each pid, pick query_per_id images for QUERY (camid+offset), rest go to GALLERY (original camid)
    moved_back_to_train = []
    for pid in sorted(test_pids):
        items = pid_to_imgs[pid]  # list of (path, pid, camid)
        if len(items) < (query_per_id + 1):  # need >= 2 images per pid for eval
            moved_back_to_train.append(pid)
            continue

        idxs = list(range(len(items)))
        random.shuffle(idxs)
        q_idxs = set(idxs[:query_per_id])  # choose query images

        for i, (p, pid_i, cam) in enumerate(items):
            if i in q_idxs:
                # IMPORTANT: make query camid different from gallery camid
                query_data.append((p, pid_i, cam + QUERY_CAM_OFFSET))
            else:
                gallery_data.append((p, pid_i, cam))

    # If some IDs had too few images, move them to train (and relabel)
    if moved_back_to_train:
        print(f"Info: {len(moved_back_to_train)} IDs had <2 images; moving them to train.")
        for pid in moved_back_to_train:
            # add to train and relabel
            for (p, _, c) in pid_to_imgs[pid]:
                train_set_data.append((p, None, c))  # placeholder, relabel next
        # relabel again
        unique_train_pids = sorted({pid for _, pid, _ in train_set_data if pid is not None})
        # Add newly moved pids at the end
        next_id = len(unique_train_pids)
        # Map old pid -> new contiguous id
        pid_remap = {}
        i = 0
        for pid in sorted({pid for _, pid, _ in train_set_data if pid is not None}):
            pid_remap[pid] = i
            i += 1
        # assign new IDs for placeholders (None) in the order they appear by original pid
        for idx, (p, pid, c) in enumerate(train_set_data):
            if pid is None:
                # get original pid from path by searching pid_to_imgs
                # (slow path but safe; could store alongside above if needed)
                # We'll infer pid from folder name: robust, since path contains identity dir name.
                ident_dir = os.path.basename(os.path.dirname(p))
                orig_pid = pid_to_id.get(ident_dir, None)
                if orig_pid is None:
                    # fallback: put into class 0
                    new_id = 0
                else:
                    if orig_pid not in pid_remap:
                        pid_remap[orig_pid] = i
                        i += 1
                    new_id = pid_remap[orig_pid]
                train_set_data[idx] = (p, new_id, c)

    # Sanity checks
    def uniq_pids(lst): return np.unique([it[1] for it in lst]) if len(lst) else np.array([])
    print("\nData Split Summary:")
    print(f"- Training identities: {len(uniq_pids(train_set_data))}\n- {uniq_pids(train_set_data)}")
    print(f"- Query identities: {len(uniq_pids(query_data))}\n- {uniq_pids(query_data)}")
    print(f"- Gallery identities: {len(uniq_pids(gallery_data))}\n- {uniq_pids(gallery_data)}")

    if any(pid not in [g[1] for g in gallery_data] for pid in [q[1] for q in query_data]):
        raise AssertionError("The query set contains identities not found in the gallery set.")

    # 4) Define custom dataset class
    class BadmintonReID(torchreid.data.ImageDataset):
        def __init__(self, root='', **kwargs):
            self.train = train_set_data
            self.query = query_data
            self.gallery = gallery_data
            super(BadmintonReID, self).__init__(self.train, self.query, self.gallery, **kwargs)

    # Register once
    if register_dataset:
        torchreid.data.register_image_dataset(dataset_name, BadmintonReID)

    # Choose a safe num_instances for RandomIdentitySampler
    imgs_per_train_pid = defaultdict(int)
    for _, pid, _ in train_set_data:
        imgs_per_train_pid[pid] += 1
    min_imgs_per_pid = min(imgs_per_train_pid.values()) if imgs_per_train_pid else 1
    num_instances = max(2, min(4, min_imgs_per_pid))  # cap at 4, at least 2

    # IMPORTANT: root should be a parent folder (not the custom dataset dir) if you also want Market1501.
    # Start with only your dataset; add Market1501 later after verifying pipeline.
    data_root = "/content"  # a common root
    # Make sure torchreid can find your custom dataset by name:
    # It will instantiate BadmintonReID (which uses the prebuilt lists above).
    datamanager = torchreid.data.datamanager.ImageDataManager(
        root=data_root,
        sources=[dataset_name],          # start with custom only; add 'market1501' when ready
        targets=dataset_name,
        height=256,
        width=256,
        transforms=['random_flip', 'random_crop', 'color_jitter', 'normalize'],
        num_instances=num_instances,
        workers=2,
        train_sampler='RandomIdentitySampler',
    )

    print(f"\nDataManager initialized:")
    print(f"- Training identities: {datamanager.num_train_pids}")

    # Build model (use softmax head so the TripletEngine can combine CE + Triplet)
    model = torchreid.models.build_model(
        name='osnet_x1_0',
        num_classes=datamanager.num_train_pids,
        loss='triplet',
        pretrained=True,
        use_gpu=torch.cuda.is_available()
    )

    if torch.cuda.is_available():
        model = model.cuda()
        print("Moving model to GPU...")
    else:
        print("Warning: Using CPU for model...")

    # Optimizer & scheduler
    optimizer = torchreid.optim.build_optimizer(
        model,
        optim='adam',
        lr=3e-4,
        weight_decay=5e-4
    )
    scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=[20, 40]
    )

    engine = torchreid.engine.ImageTripletEngine(
        datamanager,
        model,
        optimizer=optimizer,
        scheduler=scheduler,
        margin=0.3,
        weight_t=1.0,
        weight_x=1.0,
        label_smooth=True
    )

    # Train
    output_dir = "/content/badminton_reid_model"
    os.makedirs(output_dir, exist_ok=True)

    engine.run(
        save_dir=output_dir,
        max_epoch=1,       # bump this once pipeline works (you had 1)
        eval_freq=10,
        print_freq=10,
        test_only=False,
    )

    # Evaluate + visualize rank
    print("\nPerforming visual ranking...")
    engine.run(
        save_dir=output_dir,
        max_epoch=0,
        test_only=True,
        visrank=True,
        visrank_topk=5,
        use_metric_cuhk03=False,
        ranks=[1, 5, 10],
        rerank=False
    )

    print(f"Training completed! Model saved to: {output_dir}")
    return output_dir


In [8]:
def setup_custom_strongsort_config():
    """Create custom StrongSORT configuration for badminton"""

    import yaml

    # Custom configuration for badminton tracking
    strongsort_config = {
        'tracker_type': 'strongsort',
        'reid_weights': '/content/badminton_reid_model/model.pth.tar',
        'max_dist': 0.2,           # Maximum distance for matching
        'min_confidence': 0.3,      # Minimum detection confidence
        'nms_max_overlap': 0.7,     # Non-maximum suppression
        'max_iou_distance': 0.7,    # Maximum IoU distance
        'max_age': 30,              # Frames to keep lost tracks
        'n_init': 3,                # Frames to confirm track
        'nn_budget': 100,           # Feature budget for ReID
        'ema_alpha': 0.9,           # Exponential moving average for features
    }

    # Save configuration
    config_path = "/content/badminton_strongsort.yaml"
    with open(config_path, 'w') as f:
        yaml.dump(strongsort_config, f)

    print(f"Custom configuration saved to: {config_path}")
    return config_path

In [9]:
def run_custom_strongsort(yolo_model_path, reid_model_path, test_video_path):
    """Run StrongSORT with custom trained ReID model"""

    # Setup custom configuration
    config_path = setup_custom_strongsort_config()

    # Run tracking with custom ReID model
    output_dir = "runs/track/custom_badminton"

    !python track.py \
        --yolo-model {yolo_model_path} \
        --reid-model {reid_model_path} \
        --tracking-method strongsort \
        --source {test_video_path} \
        --imgsz 640 \
        --conf-thres 0.4 \
        --iou-thres 0.5 \
        --max-det 300 \
        --device 0 \
        --save \
        --save-txt \
        --save-conf \
        --project runs/track \
        --name custom_badminton \
        --exist-ok

    print(f"Custom tracking completed! Results saved to: {output_dir}")

In [10]:
"""Execute the complete pipeline step by step"""

print("=== BADMINTON PLAYER TRACKING PIPELINE ===\n")

# Step 1: Verify YOLOv8 model
yolo_model_path = "/content/drive/MyDrive/FIT3163,3164/YOLO/dataset_2/runs/detect/train5/weights/best.pt"

=== BADMINTON PLAYER TRACKING PIPELINE ===



In [11]:
# Step 2: Process annotations
print("\n🔄 Step 2: Processing annotations...")
process_all_videos(reid_base_dir="/content/drive/MyDrive/FIT3163,3164/REID/train")


🔄 Step 2: Processing annotations...

Processing /content/drive/MyDrive/FIT3163,3164/REID/train/phua_1...
Processing video: /content/drive/MyDrive/FIT3163,3164/REID/train/phua_1/phua_vid_1.mp4
Using annotations: /content/drive/MyDrive/FIT3163,3164/REID/train/phua_1/gt/gt.txt
Created directory: /content/reid_training_data/phua_vid_1_player_001
Created directory: /content/reid_training_data/phua_vid_1_player_002
Created directory: /content/reid_training_data/phua_vid_1_player_003
Created directory: /content/reid_training_data/phua_vid_1_player_004
Processed 100 frames, extracted 396 crops
Processed 200 frames, extracted 796 crops
Completed! Total crops extracted: 1076
Player 1: 269 crops in /content/reid_training_data/phua_vid_1_player_001
Player 2: 269 crops in /content/reid_training_data/phua_vid_1_player_002
Player 3: 269 crops in /content/reid_training_data/phua_vid_1_player_003
Player 4: 269 crops in /content/reid_training_data/phua_vid_1_player_004

Processing /content/drive/MyDriv

In [14]:
# Step 3: Train ReID model
print("\n🧠 Step 3: Training ReID model...")
reid_model_dir = train_badminton_reid_model(
    register_dataset=True,
    dataset_name='badminton1'
    )


🧠 Step 3: Training ReID model...
Training data summary:
- Total identities: 72:
0. bryan_vid_1_player_001
1. bryan_vid_1_player_002
2. bryan_vid_1_player_003
3. bryan_vid_1_player_004
4. phua_vid_1_player_001
5. phua_vid_1_player_002
6. phua_vid_1_player_003
7. phua_vid_1_player_004
8. phua_vid_2_player_001
9. phua_vid_2_player_002
10. phua_vid_2_player_003
11. phua_vid_2_player_004
12. phua_vid_3_player_001
13. phua_vid_3_player_002
14. phua_vid_3_player_003
15. phua_vid_3_player_004
16. phua_vid_4_player_001
17. phua_vid_4_player_002
18. phua_vid_4_player_003
19. phua_vid_4_player_004
20. thom_vid_1_player_001
21. thom_vid_1_player_002
22. thom_vid_1_player_003
23. thom_vid_1_player_004
24. thom_vid_2_player_001
25. thom_vid_2_player_002
26. thom_vid_2_player_003
27. thom_vid_2_player_004
28. thom_vid_3_player_001
29. thom_vid_3_player_002
30. thom_vid_3_player_003
31. thom_vid_3_player_004
32. ting_vid_10_player_001
33. ting_vid_10_player_002
34. ting_vid_10_player_003
35. ting_vid

In [15]:
# Step 4: Test custom tracking
print("\n🎯 Step 5: Testing custom tracking...")
reid_model_path = "/content/badminton_reid_model/model/model.pth.tar-1"
run_custom_strongsort(yolo_model_path, reid_model_path, "test_video.mp4")

print("\n🎉 Pipeline setup complete!")


🎯 Step 5: Testing custom tracking...
Custom configuration saved to: /content/badminton_strongsort.yaml
python3: can't open file '/content/track.py': [Errno 2] No such file or directory
Custom tracking completed! Results saved to: runs/track/custom_badminton

🎉 Pipeline setup complete!


In [16]:
# Import all necessary libraries for the script to be self-contained
import os
import yaml
import argparse
from functools import partial
from pathlib import Path
import cv2
import torch
import sys
from types import SimpleNamespace

# Add the parent directory of boxmot to the system path to allow imports
# This is necessary because the files are not in a standard Python package
# In a real-world scenario, you would install boxmot as a package
# We'll assume the boxmot directory is a sibling to the current script's location.
# If this causes issues, you might need to adjust the path.
try:
    from boxmot.tracker_zoo import create_tracker
    from boxmot.utils import TRACKER_CONFIGS, WEIGHTS
    from boxmot.utils.checks import RequirementsChecker
    from boxmot.engine.detectors import default_imgsz, get_yolo_inferer, is_ultralytics_model
    from boxmot import TRACKERS
    from ultralytics import YOLO
    from ultralytics.utils.plotting import Annotator, colors
    from ultralytics.utils import plotting
    # Make every drawing call a no-op to disable default plotting
    plotting.Annotator.box       = lambda *args, **kwargs: None
    plotting.Annotator.box_label = lambda *args, **kwargs: None
    plotting.Annotator.line      = lambda *args, **kwargs: None
except ImportError as e:
    print(f"Error importing boxmot modules: {e}")
    print("Please ensure you have boxmot and ultralytics installed.")
    sys.exit()

def setup_custom_strongsort_config(reid_model_path):
    """
    Create custom StrongSORT configuration for badminton.

    Args:
        reid_model_path (str): Path to the trained ReID model weights.

    Returns:
        str: Path to the generated configuration file.
    """
    strongsort_config = {
        'reid_weights': reid_model_path,
        'max_dist': 0.2,
        'min_confidence': 0.3,
        'nms_max_overlap': 0.7,
        'max_iou_distance': 0.7,
        'max_age': 30,
        'n_init': 3,
        'nn_budget': 100,
        'ema_alpha': 0.9,
    }

    config_path = "/content/badminton_strongsort.yaml"
    with open(config_path, 'w') as f:
        yaml.dump(strongsort_config, f)

    print(f"Custom configuration saved to: {config_path}")
    return config_path

def on_predict_start(predictor, persist=False):
    """
    Initialize trackers for object tracking during prediction.
    Args:
        predictor (object): The predictor object to initialize trackers for.
        persist (bool, optional): Whether to persist the trackers if they already exist. Defaults to False.
    """
    assert predictor.custom_args.tracking_method in TRACKERS, \
        f"'{predictor.custom_args.tracking_method}' is not supported. Supported ones are {TRACKERS}"

    tracking_config = TRACKER_CONFIGS / (predictor.custom_args.tracking_method + '.yaml')
    trackers = []
    for i in range(predictor.dataset.bs):
        tracker = create_tracker(
            predictor.custom_args.tracking_method,
            tracking_config,
            predictor.custom_args.reid_model,
            predictor.device,
            predictor.custom_args.half,
            predictor.custom_args.per_class,
        )
        trackers.append(tracker)

    predictor.trackers = trackers

# callback to plot trajectories on each frame
def plot_trajectories(predictor):
    # predictor.results is a list of Results, one per frame in the batch
    for i, result in enumerate(predictor.results):
        tracker = predictor.trackers[i]
        # This function might cause issues as it's not a standard boxmot function,
        # and the original code has it defined for a different purpose.
        # We'll comment it out to avoid errors, as the main goal is to save results.
        # result.orig_img = tracker.plot_results(result.orig_img, predictor.custom_args.show_trajectories)
        cv2.waitKey(1)

@torch.no_grad()
def run_track(args):
    """
    Runs the tracking pipeline using YOLO and StrongSORT.
    This function is a reimplementation of the user's `track()` function
    but is called programmatically.
    """
    if args.imgsz is None:
        args.imgsz = default_imgsz(args.yolo_model)
    yolo = YOLO(
        args.yolo_model if is_ultralytics_model(args.yolo_model) else "yolov8n.pt",
    )

    results = yolo.track(
        source=args.source,
        conf=args.conf,
        iou=args.iou,
        agnostic_nms=args.agnostic_nms,
        show=True,
        stream=True,
        device=args.device,
        show_conf=args.show_conf,
        save_txt=args.save_txt,
        show_labels=args.show_labels,
        save=args.save,
        # verbose=args.verbose,
        exist_ok=args.exist_ok,
        project=args.project,
        name=args.name,
        classes=args.classes,
        imgsz=args.imgsz,
        vid_stride=args.vid_stride,
        line_width=args.line_width,
        save_crop=args.save_crop,
    )

    yolo.add_callback("on_predict_start", partial(on_predict_start, persist=True))
    # yolo.add_callback("on_predict_postprocess_end", plot_trajectories) # Commented out to prevent errors

    if not is_ultralytics_model(args.yolo_model):
        m = get_yolo_inferer(args.yolo_model)
        yolo_model = m(
            model=args.yolo_model,
            device=yolo.predictor.device,
            args=yolo.predictor.args,
        )
        yolo.predictor.model = yolo_model

        if not is_ultralytics_model(args.yolo_model):
            yolo.add_callback(
                "on_predict_batch_start", lambda p: yolo_model.update_im_paths(p)
            )
            yolo.predictor.preprocess = lambda imgs: yolo_model.preprocess(im=imgs)
            yolo.predictor.postprocess = lambda preds, im, im0s: yolo_model.postprocess(
                preds=preds, im=im, im0s=im0s
            )

    yolo.predictor.custom_args = args

    for _ in results:
        pass

def evaluate_badminton_tracking(yolo_model_path, reid_model_path, test_video_path):
    """
    Main function to orchestrate the tracking evaluation.

    Args:
        yolo_model_path (str): Path to the YOLO detection model.
        reid_model_path (str): Path to the trained ReID model.
        test_video_path (str): Path to the test video file.
    """
    # Create the custom configuration file
    config_path = setup_custom_strongsort_config(reid_model_path)

    # Create a dummy arguments object to pass to the track function
    # This mimics the arguments a user would provide from the command line
    args = SimpleNamespace(
        yolo_model=yolo_model_path,
        reid_model=reid_model_path,
        tracking_method='strongsort',
        source=test_video_path,
        imgsz=640,
        conf=0.4,
        iou=0.5,
        agnostic_nms=False,
        show_conf=False,
        show_labels=True,
        save=True,
        save_txt=True,
        save_conf=True,
        project="runs/track",
        name="custom_badminton",
        exist_ok=True,
        device='0' if torch.cuda.is_available() else 'cpu',
        classes=None, # None means track all classes
        vid_stride=1,
        line_width=None,
        save_crop=False,
        half=True, # Use half-precision if available
        per_class=False,
        show_trajectories=True, # Note: this feature might not work as intended in the current setup
    )

    # Run the tracking
    run_track(args)

    print(f"\nEvaluation completed!")
    print(f"Results saved to: {args.project}/{args.name}")
    print("Look for the output video and the tracking .txt files in this directory.")

if __name__ == '__main__':
    # Define the paths to your models and test video
    YOLO_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/YOLO/my_yolov8_1.pt"
    REID_MODEL_PATH = "/content/badminton_reid_model/model/model.pth.tar-1"
    TEST_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/test/reid_test_vid_1.mp4"

    evaluate_badminton_tracking(YOLO_MODEL_PATH, REID_MODEL_PATH, TEST_VIDEO_PATH)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error importing boxmot modules: No module named 'boxmot'
Please ensure you have boxmot and ultralytics installed.
Traceback (most recent call last):
  File "/tmp/ipython-input-3097829601.py", line 18, in <cell line: 0>
    from boxmot.tracker_zoo import create_tracker
ModuleNotFoundError: No module named 'boxmot'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3097829601.py", line 33, in <cell line: 0>
    sys.exit()
SystemExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: object of type 'NoneType' has no len()